In [1]:
import os
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.utils import save_image, make_grid

In [ ]:
# Параметры
IMAGE_SIZE = 64
BATCH = 128
LATENT_DIM = 256
NUM_EPOCHS = 80
LR = 2e-4
EMBED_DIM = 128   
BASE_CHANNELS = 64
BETA = 1.0     
SAMPLE_PER_CLASS = 6

In [ ]:
DATA_ROOT = Path("data/animals")
OUT_DIR = Path("outputs_cvae")
OUT_DIR.mkdir(parents=True, exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cuda


In [ ]:
# DataLoader
tf = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])  
])

dataset = datasets.ImageFolder(root=str(DATA_ROOT), transform=tf)
if len(dataset) == 0:
    raise RuntimeError(f"No images found in {DATA_ROOT}")

class_names = dataset.classes
NUM_CLASSES = len(class_names)
print(f"Found {len(dataset)} images, {NUM_CLASSES} classes:", class_names)

loader = DataLoader(dataset, batch_size=BATCH, shuffle=True, num_workers=4, pin_memory=(DEVICE.type=="cuda"), drop_last=True)

Found 13474 images, 5 classes: ['cat', 'dog', 'elephant', 'horse', 'lion']


In [ ]:
# Утилиты
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
        if m.weight is not None:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        if m.weight is not None:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        if m.weight is not None:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)

def save_image_grid(tensor, path, nrow=8):
    tensor = tensor.clamp(-1, 1)
    save_image((tensor + 1) / 2, path, nrow=nrow)

In [ ]:
# CVAE model (Encoder + Decoder + reparam)
class Encoder(nn.Module):
    def __init__(self, img_channels=3, base=BASE_CHANNELS, embed_dim=EMBED_DIM, n_classes=NUM_CLASSES, latent_dim=LATENT_DIM):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(img_channels, base, 4, 2, 1), nn.BatchNorm2d(base), nn.ReLU(True),
            nn.Conv2d(base, base*2, 4, 2, 1), nn.BatchNorm2d(base*2), nn.ReLU(True),
            nn.Conv2d(base*2, base*4, 4, 2, 1), nn.BatchNorm2d(base*4), nn.ReLU(True),
            nn.Conv2d(base*4, base*8, 4, 2, 1), nn.BatchNorm2d(base*8), nn.ReLU(True)
        )
        feat_dim = base*8*4*4
        self.label_emb = nn.Embedding(n_classes, embed_dim)

        self.fc_mu = nn.Linear(feat_dim + embed_dim, latent_dim)
        self.fc_logvar = nn.Linear(feat_dim + embed_dim, latent_dim)

    def forward(self, x, labels):
        b = x.size(0)
        f = self.conv(x)         
        f = f.view(b, -1)        
        le = self.label_emb(labels)  
        h = torch.cat([f, le], dim=1)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, img_channels=3, base=BASE_CHANNELS, embed_dim=EMBED_DIM, n_classes=NUM_CLASSES, latent_dim=LATENT_DIM):
        super().__init__()
        self.label_emb = nn.Embedding(n_classes, embed_dim)
        self.latent_input_dim = latent_dim + embed_dim
        self.fc = nn.Linear(self.latent_input_dim, base*8*4*4)

        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(base*8, base*4, 4, 2, 1), nn.BatchNorm2d(base*4), nn.ReLU(True),
            nn.ConvTranspose2d(base*4, base*2, 4, 2, 1), nn.BatchNorm2d(base*2), nn.ReLU(True),
            nn.ConvTranspose2d(base*2, base, 4, 2, 1), nn.BatchNorm2d(base), nn.ReLU(True),
            nn.ConvTranspose2d(base, img_channels, 4, 2, 1),
            nn.Tanh()   # outputs in [-1,1]
        )

    def forward(self, z, labels):
        le = self.label_emb(labels)
        z_cond = torch.cat([z, le], dim=1)  
        x = self.fc(z_cond)
        x = x.view(-1, BASE_CHANNELS*8, 4, 4)
        x = self.deconv(x)
        return x

class ConditionalVAE(nn.Module):
    def __init__(self, n_classes=NUM_CLASSES, latent_dim=LATENT_DIM, embed_dim=EMBED_DIM):
        super().__init__()
        self.encoder = Encoder(n_classes=n_classes, latent_dim=latent_dim, embed_dim=embed_dim)
        self.decoder = Decoder(n_classes=n_classes, latent_dim=latent_dim, embed_dim=embed_dim)

    def reparameterize(self, mu, logvar):
        std = (0.5 * logvar).exp()   # exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, labels):
        mu, logvar = self.encoder(x, labels)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z, labels)
        return recon, mu, logvar

In [ ]:
# Инициализация
model = ConditionalVAE(n_classes=NUM_CLASSES, latent_dim=LATENT_DIM, embed_dim=EMBED_DIM).to(DEVICE)
model.apply(weights_init)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.5, 0.999))

# fixed noise/labels для визуализации 
fixed_z = torch.randn(NUM_CLASSES * SAMPLE_PER_CLASS, LATENT_DIM, device=DEVICE)
fixed_labels = torch.tensor([i for i in range(NUM_CLASSES) for _ in range(SAMPLE_PER_CLASS)], device=DEVICE)

In [ ]:
# loss функции и train step
def loss_function(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='mean')
    # KL divergence
    kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss, kld

def train_epoch(model, loader, optimizer, device, epoch, beta=BETA):
    model.train()
    running_recon = 0.0
    running_kld = 0.0
    pbar = tqdm(loader, desc=f"CVAE epoch {epoch+1}")
    for imgs, labels in pbar:
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        recon, mu, logvar = model(imgs, labels)
        recon_loss, kld = loss_function(recon, imgs, mu, logvar)
        loss = recon_loss + beta * kld
        loss.backward()
        optimizer.step()

        running_recon += float(recon_loss.item())
        running_kld += float(kld.item())
        pbar.set_postfix({'recon': running_recon/ (pbar.n+1), 'kld': running_kld/(pbar.n+1)})
    return running_recon/len(loader), running_kld/len(loader)

In [ ]:
# функции сохранения примеров / генерации / реконструкции
def save_reconstructions(model, data_loader, path, n=8):
    model.eval()
    imgs, labels = next(iter(data_loader))
    imgs = imgs[:n].to(DEVICE)
    labels = labels[:n].to(DEVICE)
    with torch.no_grad():
        recon, _, _ = model(imgs, labels)
    cat = torch.cat([imgs.cpu(), recon.cpu()], dim=0)
    save_image_grid(cat, path, nrow=n)

def generate_by_class(model, class_idx, n=8, save_path=None):
    model.eval()
    z = torch.randn(n, LATENT_DIM, device=DEVICE)
    labels = torch.tensor([class_idx]*n, device=DEVICE)
    with torch.no_grad():
        imgs = model.decoder(z, labels).cpu()
    if save_path:
        save_image_grid(imgs, save_path, nrow=n)
    return imgs

def sample_grid_fixed(model, fixed_z, fixed_labels, save_path):
    model.eval()
    with torch.no_grad():
        z = fixed_z.to(DEVICE)
        labels = fixed_labels.to(DEVICE)
        imgs = model.decoder(z, labels).cpu()
    save_image_grid(imgs, save_path, nrow=SAMPLE_PER_CLASS)

In [10]:
# training loop, чекпоинты, примеры
best_loss = 1e9
for epoch in range(NUM_EPOCHS):
    recon_avg, kld_avg = train_epoch(model, loader, optimizer, DEVICE, epoch, beta=BETA)
    recon_path = OUT_DIR / f"recon_epoch{epoch+1}.png"
    save_reconstructions(model, loader, recon_path, n=8)

    samples_path = OUT_DIR / f"samples_epoch{epoch+1}.png"
    sample_grid_fixed(model, fixed_z, fixed_labels, samples_path)

    # чекпоинты
    torch.save({
        'epoch': epoch+1,
        'model_state': model.state_dict(),
        'opt_state': optimizer.state_dict()
    }, OUT_DIR / f"cvae_checkpoint_epoch{epoch+1}.pth")

    print(f"Epoch {epoch+1}: recon={recon_avg:.5f}, kld={kld_avg:.5f}, saved: {recon_path.name}, {samples_path.name}")

# финальное сохранение
torch.save(model.state_dict(), OUT_DIR / "cvae_final.pth")
print("Training finished. Models and examples saved to", OUT_DIR)

CVAE epoch 1: 100%|██████████| 105/105 [00:21<00:00,  4.84it/s, recon=0.196, kld=0.151]


Epoch 1: recon=0.19074, kld=0.14655, saved: recon_epoch1.png, samples_epoch1.png


CVAE epoch 2: 100%|██████████| 105/105 [00:42<00:00,  2.49it/s, recon=0.159, kld=0.0445]


Epoch 2: recon=0.15771, kld=0.04403, saved: recon_epoch2.png, samples_epoch2.png


CVAE epoch 3: 100%|██████████| 105/105 [00:37<00:00,  2.79it/s, recon=0.152, kld=0.0396]


...

Epoch 78: recon=0.11369, kld=0.03151, saved: recon_epoch78.png, samples_epoch78.png


CVAE epoch 79: 100%|██████████| 105/105 [00:21<00:00,  4.97it/s, recon=0.115, kld=0.0317]


Epoch 79: recon=0.11398, kld=0.03138, saved: recon_epoch79.png, samples_epoch79.png


CVAE epoch 80: 100%|██████████| 105/105 [00:21<00:00,  4.90it/s, recon=0.117, kld=0.0325]


Epoch 80: recon=0.11358, kld=0.03153, saved: recon_epoch80.png, samples_epoch80.png
Training finished. Models and examples saved to outputs_cvae


In [11]:
# пример использования генерации и визуализации нескольких классов

grid_imgs = []
grid_labels = []
n_per = SAMPLE_PER_CLASS
for cls_idx in range(min(5, NUM_CLASSES)):
    imgs = generate_by_class(model, cls_idx, n=n_per)
    grid_imgs.append(imgs)
    grid_labels += [class_names[cls_idx]] * n_per

# Склеим все в один тензор и сохраним
grid = torch.cat(grid_imgs, dim=0)
save_image_grid(grid, OUT_DIR / "generation_first5_classes.png", nrow=n_per)
print("Saved generation grid for first 5 classes:", OUT_DIR / "generation_first5_classes.png")

Saved generation grid for first 5 classes: outputs_cvae\generation_first5_classes.png


In [12]:
def class_name_to_index(name):
    lowered = [c.lower() for c in class_names]
    name_l = name.lower()
    if name_l in lowered:
        return lowered.index(name_l)
    for i,c in enumerate(lowered):
        if c.startswith(name_l) or name_l in c:
            return i
    raise ValueError("class not found")

def generate_by_name(name, n=6, save_path=None):
    idx = class_name_to_index(name)
    imgs = generate_by_class(model, idx, n=n, save_path=save_path)
    return imgs

In [14]:
generate_by_name("dog", n = 10, save_path=OUT_DIR / "generation_dog.png")


tensor([[[[ 0.0124, -0.0080, -0.0059,  ..., -0.0971, -0.1025, -0.0863],
          [ 0.0140,  0.0281,  0.0162,  ..., -0.1067, -0.1138, -0.0859],
          [ 0.0371,  0.0512,  0.0413,  ..., -0.1023, -0.1044, -0.0994],
          ...,
          [-0.7692, -0.8026, -0.7991,  ..., -0.2506, -0.2410, -0.2230],
          [-0.7388, -0.7971, -0.7912,  ..., -0.2435, -0.2424, -0.2243],
          [-0.6510, -0.7470, -0.7439,  ..., -0.2277, -0.2386, -0.1950]],

         [[ 0.0254,  0.0524,  0.0460,  ..., -0.0526, -0.0622, -0.0706],
          [ 0.0718,  0.0880,  0.0693,  ..., -0.0501, -0.0564, -0.0588],
          [ 0.0763,  0.1024,  0.1119,  ..., -0.0303, -0.0436, -0.0678],
          ...,
          [-0.7571, -0.8004, -0.7883,  ..., -0.2419, -0.2411, -0.2256],
          [-0.7566, -0.7982, -0.8025,  ..., -0.2423, -0.2364, -0.2307],
          [-0.6594, -0.7596, -0.7627,  ..., -0.2615, -0.2501, -0.2247]],

         [[-0.0440, -0.0457, -0.0523,  ..., -0.1836, -0.1927, -0.1924],
          [-0.0232, -0.0304, -